In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
cd workspace/theislab/mubind/notebooks/batch

/mnt/c/Users/ignacio.ibarra/Dropbox/workspace/theislab/mubind/notebooks/batch


In [4]:
import mubind as mb
import numpy as np
import pandas as pd
import torch
import bindome as bd
bd.constants.ANNOTATIONS_DIRECTORY = '../../annotations'
# mb.models.MultiBind
import torch.optim as topti
import torch.utils.data as tdata
import matplotlib.pyplot as plt
import logomaker

# Use a GPU if available, as it should be faster.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

/home/rio/miniconda3/envs/mubind/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda:0


In [5]:
import os

n_sample = 5000
df = []
basedir = '../../../mubind-pipeline/output/snakemake/SELEX'
batch = 0
for tf in os.listdir(basedir):
    for f in os.listdir(os.path.join(basedir, tf)):
        if f.endswith('.tsv.gz'):
            df2 = pd.read_csv(os.path.join(basedir, tf, f), sep='\t', index_col=0) # .head(100)
            print(f, df2.shape)
            assert 'batch' in df2
            # print(df2.columns)
            # df2 = df2.sample(100000)
            n_rounds = len(df2.columns) - 2
            df2.columns =  list(range(n_rounds)) + ['batch', 'is_count_data']
            df2['batch'] = batch
            df2['n_rounds'] = n_rounds
            # df2 = mb.pp.sample_rounds(df2, n_rounds, n_sample_per_round)
            df2 = df2.sample(n_sample)
            print(df2.shape)
            batch += 1
            df.append(df2)
            # assert False



GATA4-ZeroCycle_0_R46777-R46777_10000000.tsv.gz (6092172, 6)
(5000, 7)
GATA4TBX5-ZeroCycle_0_R46999-R46999_10000000.tsv.gz (6708536, 6)
(5000, 7)
NKX2-5-ZeroCycle_0_R46777-R46777_10000000.tsv.gz (6193797, 6)
(5000, 7)
NKX2-5GATA4-ZeroCycle_0_R46777-R46777_10000000.tsv.gz (5707797, 6)
(5000, 7)
NKX2-5TBX5-ZeroCycle_0_R46888-R46888_10000000.tsv.gz (7349709, 6)
(5000, 7)


In [6]:
# concatenation/reordering
df = pd.concat(df)
df = df[[c for c in df.columns if not c in ['batch', 'is_count_data', 'n_rounds']] + ['batch', 'is_count_data', 'n_rounds']]

In [7]:
df.shape

(25000, 7)

In [8]:
print(df.shape)
dataset = mb.datasets.SelexDataset(df, n_rounds=df['n_rounds'], labels=list(df.columns[:-3]))
train = tdata.DataLoader(dataset=dataset,
                         batch_size=1024,
                         # batch_size=512,
                         shuffle=False)

(25000, 7)


In [9]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
import numpy as np

In [11]:
%load_ext line_profiler


In [12]:
import gc
gc.collect()

80

In [13]:
print('total entries', dataset.rounds.shape)

total entries (25000, 4)


In [14]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.expand_frame_repr', False)


In [16]:
# assign batch and data type

# data['batch'] = 0
# data['is_count_data'] = 1
# n_batches=3

dirichlet_regularization = 0
# for dirichlet_regularization_log in range(-5, 3):

n_epochs = 450
log_each = 25
n_kernels = 5
lr = [0.05] * n_kernels
wd = [0.05,] + [0.005] * (n_kernels - 1)
early_stopping = [5,] + [25] * (n_kernels - 1)
w = 15 # min(len(motif), 6)
opt_kernel_shift = 1
opt_kernel_length = 1

from matplotlib import rcParams
rcParams['figure.figsize'] = 8, 1

import time
t0 = time.time()

criterion = mb.tl.PoissonLoss()

for ignore_kernel in [1]: # [0, 1]:
    print('\n\n##NEW JOB')
    print('ignore kernel', ignore_kernel)
    model_by_k, res_next = mb.tl.optimize_iterative(train, device, w=w, show_logo=0, criterion=criterion,
                                                 opt_kernel_shift=opt_kernel_shift, opt_kernel_length=opt_kernel_length,
                                                 dirichlet_regularization=dirichlet_regularization, # 10 ** dirichlet_regularization_log,
                                                 lr=lr, weight_decay=wd, ignore_kernel=ignore_kernel, use_mono=True,
                                                 num_epochs=n_epochs, early_stopping=early_stopping, use_dinuc=True, # optimiser=torch.optim.LBFGS,
                                                 n_kernels=n_kernels, log_each=log_each, stop_at_kernel=None) #  seed=seed) # seeds.index[0]) #
    print('##DONE....\n\n')


print('total time: %.3f s' % ((time.time() - t0)))
# res = []
# model_by_k = {}

model = model_by_k





##NEW JOB
ignore kernel 1
next w 15 <class 'int'>
# rounds {4}
# rounds {4}
# use_mono True
# use_dinuc True
# batches 5
# kernels 5
# initial w 15
# enr_series True

Kernel to optimize 0

FREEZING KERNELS
optimizing feature type mono
setting grad status of kernel (mono, dinuc) at 0 to (1, 0)
setting grad status of kernel (mono, dinuc) at 1 to (0, 0)
setting grad status of kernel (mono, dinuc) at 2 to (0, 0)
setting grad status of kernel (mono, dinuc) at 3 to (0, 0)
setting grad status of kernel (mono, dinuc) at 4 to (0, 0)


kernels mask [0 1 1 1 1]
optimizer:  <class 'torch.optim.adam.Adam'> 
criterion: <class 'mubind.tl.loss.PoissonLoss'> 
# epochs: 450 
early_stopping: 5
lr= 0.05, weight_decay= 0.05, dir weight= 0
Epoch: 26, Loss: 2.370995 , best epoch: 24 secs per epoch: 0.450 s
Epoch: 51, Loss: 2.370834 , best epoch: 49 secs per epoch: 0.432 s
Epoch: 76, Loss: 2.370784 , best epoch: 74 secs per epoch: 0.425 s
Epoch: 101, Loss: 2.370763 , best epoch: 99 secs per epoch: 0.424 s
E

KeyboardInterrupt: 

### We can visualize the overall results obtained by the network, once the training is finished

In [ ]:
# %load_ext line_profiler
# %lprun -f mb.tl.optimize_simple mb.tl.optimize_iterative(train, device, w=w, show_logo=0, criterion=criterion, opt_kernel_shift=0, opt_kernel_length=0, dirichlet_regularization=dirichlet_regularization, lr=lr, weight_decay=wd, ignore_kernel=ignore_kernel, num_epochs=n_epochs, early_stopping=early_stopping, use_dinuc=False, n_kernels=n_kernels, log_each=log_each, stop_at_kernel=None) #  seed=seed) # seeds.index[0]) #


In [ ]:
import mubind as mb

In [ ]:
model = model_by_k
mb.tl.scores(model, train)


In [ ]:
counts = mb.tl.predict(model, train)
counts

In [ ]:
weight_mono_i = model.binding_modes.conv_mono[1].weight
pos_w_sum = weight_mono_i[weight_mono_i > 0].sum()

In [ ]:
weight_mono_i[weight_mono_i > 0]

In [ ]:
rcParams['figure.figsize'] = 5, 5
mb.pl.kmer_enrichment(model, train, log_scale=False, style='scatter', ylab='t1', xlab='p1', k=5)


In [ ]:
mb.pl.conv(model)
